In [4]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.spatial as spatial


In [3]:

def calculate_slope(deltax_deltay):
    '''
    Parameters
    ----------
    deltax_deltay : 1 x array-like. [X2-X2, Y2-Y1 ]

    Returns
    -------
    vertical : boolean. True if the line is vertically oriented (deltaX==0)

    slope : float. 
    
    '''
    zeros_present = tuple((deltax_deltay == 0).tolist())

    if sum(zeros_present)>0:
        return(slope_dict[zeros_present])
    
    else:
        return(False, deltax_deltay[1]/float(deltax_deltay[0]))

slope_dict = {(True,False) : (True, np.nan) ,# xchanges y doesnt
              (False, True): (False, 0.0), #y doesnt, x changes
              (True, True): Exception('Zero slopes for both not possible!!')}
        
     
    

def solve_for_intercept(x1y1, x2y2,m):
    '''
    '''
    x1,y1 = x1y1
    x2,y2 = x2y2
    
    c = (y1 + y2 - m*x1 - m*x2)/2.0
    return(c)
    
    

    

def make_rectangle_between_2_points(A, B, **kwargs):
    '''First calculate the relative coordinates of B wref to A. 
    Then draw a straight line between 0,0 and B_rel and 'undo' the
    slope. To this vertical line not apply rectangular bands on left and right.
    
    Output bottom left and top right vertices along with the rotation matrix
    along with the rotation matrix used to undo the slope of the 0,0-B_rel line
    for application to other poitns. 

    Parameters
    ----------
    A, B : 1x2 array like. xy coordinates of start(A) and end(B) points

    Keyword Arguments
    -----------------
    rectangle_width : float>0. The width of the rectangle between A and B. 

    Returns
    -------
    corner_limits : tuple.
                    Consisting of x0,x1,y0,y1

    '''
    # treat A as origin, calculate slope between B and A
    B_rel = B-A 
    # 'un-rotate' B and thus form a vertical rectangle easily
    theta = np.arctan2(B_rel[1], B_rel[0])

    #theta_tobe_rotated = np.remainder(theta, np.pi/2)
    theta_tobe_rotated = np.pi/2.0 - theta
    rotation_matrix = rot_mat(theta_tobe_rotated)
    B_rotated = np.dot(rotation_matrix, B_rel)
    x0, x1 = -kwargs['rectangle_width']*0.5, kwargs['rectangle_width']*0.5
    y0, y1 = 0, B_rotated[1]

    return([x0,x1,y0,y1], rotation_matrix)
	
def get_points_in_rectangle(corner_limits, startpt,
                            many_points, rotn_matrix):
    ''' Many points are checked if they are within 
    the rectangle defined by the bottom left point (x0,y0) and 
    the top right corner (x1,y1).

    Corner limits is a tuple with four entries (x0,x1,y0,y1)
    x0,x1 the x coordinates defining the width of the rectangle
    y0,y1 the y coordinates defininf the height of the rectanlge
   
    rotn_matrix is the rotation matrix to rotate the many points into 
    the same frame of reference as the rectangle. it is in the 
    form of a 2 x 2 array with the form described 
    [here](https://en.wikipedia.org/wiki/Rotation_matrix)



    '''
    x0,x1,y0,y1 = corner_limits
    relative_posns = many_points - startpt
    rotated_pts = np.apply_along_axis(dot_product_for_rows, 1, relative_posns,
                                      rotn_matrix)
    
    within_x = np.logical_and(rotated_pts[:,0] >= np.min([x0,x1]),
                              rotated_pts[:,0] <= np.max([x1,x0]))

    within_y = np.logical_and(rotated_pts[:,1] >= np.min([y0,y1]),
                              rotated_pts[:,1] <= np.max([y0,y1]))
    
    within_pts = np.logical_and(within_x, within_y)
    return(many_points[within_pts])

def dot_product_for_rows(xy_row, rotation_matrix):
    return(np.dot(rotation_matrix, xy_row))

def dot_product_w_sum(xy_row, rotation_matrix):
    return(np.sum(rotation_matrix*xy_row, 1))


def rot_mat(theta):
    rotation_matrix = np.float32(np.row_stack(([np.cos(theta), -np.sin(theta)],
                                    [np.sin(theta), np.cos(theta)])))
    return(rotation_matrix)

In [ ]:
def get_points_in_between_thecircleversion(start_point, end_point, 
                                          all_other_points,**kwargs):
    '''Take 2 at getting perhaps a faster version of the 
    previous get_points_in_between function. 
    
    It is fast *and* dirty ... and doesn't quite apply when many bats are packed tightly
    together ... as long as the 'rectangle_width' is decently large -- then it
    should be okay..

    
    '''
    # get line equation from A to B
    diff_x_y = end_point-start_point
    vertical, m = calculate_slope(diff_x_y)
            
    numpoints = 100 # choose a default density for now 
    
    points_along_line = np.zeros((numpoints, 2))
    
    if not vertical:
        points_along_line[:,0] = np.linspace(start_point[0],end_point[0],
                                                 numpoints) # x coordinates
        c = solve_for_intercept(start_point,end_point,m)
        points_along_line[:,1] = m*points_along_line[:,0] + c # y coordinates 
    
    else:
        points_along_line[:,0] = start_point[0] # x coordinates
        points_along_line[:,1] =  np.linspace(start_point[1], end_point[1],
                                 numpoints)# y coordinates 
    
    # get the distance from each of the points to all other points
    distance_from_line = spatial.distance_matrix(points_along_line, 
                                                         all_other_points)
    within_r_dist_from_line = distance_from_line<= kwargs['rectangle_width']

    point_ids = np.argwhere(np.any(within_r_dist_from_line, axis=0))
    return(all_other_points[point_ids])
        

In [5]:
def make_rectangle_between_2_points(A, B, **kwargs):
    '''First calculate the relative coordinates of B wref to A. 
    Then draw a straight line between 0,0 and B_rel and 'undo' the
    slope. To this vertical line not apply rectangular bands on left and right.
    
    Output bottom left and top right vertices along with the rotation matrix
    along with the rotation matrix used to undo the slope of the 0,0-B_rel line
    for application to other poitns. 

    Parameters
    ----------
    A, B : 1x2 array like. xy coordinates of start(A) and end(B) points

    Keyword Arguments
    -----------------
    rectangle_width : float>0. The width of the rectangle between A and B. 

    Returns
    -------
    corner_limits : tuple.
                    Consisting of x0,x1,y0,y1

    '''
    # treat A as origin, calculate slope between B and A
    B_rel = B-A 
    # 'un-rotate' B and thus form a vertical rectangle easily
    theta = np.arctan2(B_rel[1], B_rel[0])

    #theta_tobe_rotated = np.remainder(theta, np.pi/2)
    theta_tobe_rotated = np.pi/2.0 - theta
    rotation_matrix = rot_mat(theta_tobe_rotated)
    B_rotated = np.dot(rotation_matrix, B_rel)
    x0, x1 = -kwargs['rectangle_width']*0.5, kwargs['rectangle_width']*0.5
    y0, y1 = 0, B_rotated[1]

    return([x0,x1,y0,y1], rotation_matrix)

In [11]:
a=np.array([0,0]); b=np.array([1,1])

make_rectangle_between_2_points(a,b)

KeyError: 'rectangle_width'

In [7]:
import scipy.spatial as spatial
import numpy as np
spatial.distance_matrix(np.array([[0,0]]), np.array([[0,1],[1,0]], np.array([[1,1]])))

TypeError: Cannot construct a dtype from an array